In [110]:


def BpL(pre, edge, i):
    res = []
    
    for j in range(i+1,len(pre)):

        if int(pre[j]) == int(pre[j-1]) - 1 and (int(edge[j-1]) == 1 or int(edge[j-1]) == 2):
            res.append(j)
        else:
            break
    
    return res

def BpR(pre, edge, i):
    res = []
    
    for j in range(i+1,len(pre)):

        if int(pre[j]) == int(pre[j-1]) + 1 and (int(edge[j-1]) == 1 or int(edge[j-1]) == 2):
            res.append(j)
        else:
            break
    
    return res

def AL(pre, edge, i):
    res = []

    if pre[i] < pre[i-1]:
        res.append(i-1)
    else:
        return []

    j = i-1
    while j > 0:
        if (int(pre[j]) == int(pre[j-1]) - 1) and (int(edge[j-1]) == 1 or int(edge[j-1]) == 2):
            res.append(j-1)
        else:
            break
        j-=1

    return res


def AR(pre, edge, i):
    res = []

    if pre[i] > pre[i-1]:
        res.append(i-1)
    else:
        return []

    j = i-1
    while j > 0:
        if (int(pre[j]) == int(pre[j-1]) + 1) and (int(edge[j-1]) == 1 or int(edge[j-1]) == 2):
            res.append(j-1)
        else:
            break
        j-=1

    #if int(pre[res[-1]]) == int(pre[0]) + 1: #Actually does this hold?
        #res.append(0)

    return res


def elbowR(pre, edge):

    res = []
    for i in range(1,len(pre)-1):

        #Need to ensure i does not have a right child TODO

        bL = BpL(pre, edge, i) #Ensures no vertex has a right child
        ar = AR(pre, edge, i) #Ensures no vertex has a left child

        #print(f"i: {i}\nbL:{bL}\nar:{ar}\n ") 

        if (len(bL) >= 1 and #cL(i) exists.
            len(ar) >= 1 and #i=cR(p(i))
            (len(ar) == 1 or len(bL) == 1) and #One of the branches is trivial
            (int(ar[-1]) == 0 or int(edge[ar[-1]-1]) == 0) #The top node is either the root or a non-contig edge
        ):
            res.append(i)
    
    return res

def elbowL(pre, edge):
    
    res = []
    for i in range(1,len(pre)-1):

        #Need to ensure i does not have a left child TODO

        bR = BpR(pre, edge, i) #Ensures no vertex has a left child
        al = AL(pre, edge, i) #Ensures no vertex has a right child

        #print(f"i: {i}\nbL:{bR}\nar:{al}\n ") 

        if (len(bR) >= 1 and #cR(i) exists.
            len(al) >= 1 and #i=cL(p(i))
            (len(al) == 1 or len(bR) == 1) and #One of the branches is trivial
            (int(al[-1]) == 0 or int(edge[al[-1]-1]) == 0) #The top node is either the root or a non-contig edge
        ):
            res.append(i)
    
    return res

def elbow(pre, edge):
    a = elbowL(pre, edge) + elbowR(pre, edge)

    for k in a:
        edge = list(edge)
        edge[k-1] = "-"
    
    return edge

In [111]:
pre="132"
edge="11"

elbow(pre,edge)

#elbowR(pre, edge)

['-', '1']

In [112]:
pre=[4,1,3,2,5,6,9,8,7]
edge=[0,1,1,1,1,1,1,0]
i=6

print(len(BpL(pre, edge, i)) >= 1)
print(len(AR(pre, edge, i)) >= 1)
print((len(AR(pre, edge, i)) == 1 or len(BpL(pre, edge, i)) == 1))
print(int(AR(pre, edge, i)[-1]) == 0 or int(edge[AR(pre, edge, i)[-1]-1]) == 0)

elbow(pre, edge)

True
True
True
False


[0, '-', 1, 1, 1, 1, 1, 0]

In [67]:
from itertools import permutations, product

In [93]:
def get_leaf(preorder):

    if len(preorder) == 0:
        return []
    elif len(preorder) == 1:
        print(preorder)
        return preorder

    root = int(preorder[0])

    left = [x for i, x in enumerate(preorder[1:]) if int(x) < root]
    right = [x for i, x in enumerate(preorder[1:]) if int(x) > root]

    leaf = []
    leaf += get_leaf(left)
    leaf += get_leaf(right)

    return leaf


def mirrored(preorder):
    """Returns the mirrored preorder based on the reordering n+1-i"""

    n = len(preorder)
    return "".join([str(n+1-int(x)) for x in preorder])

def avoid231(preorder):
    """Returns whether the given preorder is 231 avoiding."""

    p = list(map(int, preorder))
    for i, x in enumerate(p[1:-1]):
        left = p[:i+1]
        right = p[i+2:]

        left_max = max(filter(lambda a: a < x, left), default=0)
        right_min = min(right)

        if left_max > right_min:
            return False
    
    return True

In [94]:
get_leaf("1324")

elbow("1324", "111")

['2']
['4']


'111'

In [113]:
n=4


patterns = [x for x in permutations(range(1,n+1), n)]
seen = []
p_mirrored = [x for x in patterns if x not in seen and not seen.append(mirrored(x))] #mirrored is broken
preorders = list(filter(avoid231, p_mirrored))

all_edges = [x for x in list(product([0,1,2],repeat=n-1))]
all_edges = [[item for item in pair] for pair in all_edges]


res = []

for x in preorders:
    res += [[x, []]]

    leaves = get_leaf(x)

    idxs = []
    for k in leaves:
        idxs.append(x.index(k)-1)
    
    print(idxs)
    
    for edge in all_edges:
        
        e = edge.copy()
        for i in idxs:
            e[i] = 1
        
        e = elbow(x, e)

        for j in idxs:
            e[j] = "-"



        res[-1][1].append(e)

    new = []
    [new.append(item) for item in res[-1][1] if item not in new]
    res[-1][1] = new

for k in res:

    print(str(k[0]) + ": " + str(k[1]))

[4]
[2]
[3]
[2]
[2]
[4]
[1, 2]
[3]
[2]
[2]
[2]
[1]
[4]
[0, 2]
[1]
[3]
[0, 2]
[2]
[4]
[1, 2]
[1]
[4]
[1, 2]
[3]
[2]
[2]
[2]
[1]
[3]
[1, 2]
[2]
[2]
[1]
[2]
(1, 2, 3, 4): [[0, 0, '-'], [0, 1, '-'], [0, 2, '-'], [1, 0, '-'], [1, 1, '-'], [1, 2, '-'], [2, 0, '-'], [2, 1, '-'], [2, 2, '-']]
(1, 2, 4, 3): [[0, '-', '-'], [1, '-', '-'], [2, '-', '-']]
(1, 3, 2, 4): [['-', '-', '-']]
(1, 4, 2, 3): [[0, '-', '-'], [1, 0, '-'], [1, 1, '-'], [1, 2, '-'], [2, 0, '-'], [2, 1, '-'], [2, 2, '-']]
(1, 4, 3, 2): [[0, 0, '-'], ['-', 1, '-'], ['-', 2, '-'], [1, 0, '-'], [2, 0, '-']]
(2, 1, 3, 4): [['-', 0, '-'], ['-', 1, '-'], ['-', 2, '-']]
(2, 1, 4, 3): [['-', 0, '-'], ['-', 1, '-'], ['-', 2, '-']]
(3, 1, 2, 4): [['-', '-', '-']]
(3, 2, 1, 4): [[0, '-', '-'], [1, '-', '-'], [2, '-', '-']]
(4, 1, 2, 3): [[0, 0, '-'], ['-', 1, '-'], ['-', 2, '-'], [1, 0, '-'], [2, 0, '-']]
(4, 1, 3, 2): [[0, '-', '-'], [1, 0, '-'], [1, 1, '-'], [1, 2, '-'], [2, 0, '-'], [2, 1, '-'], [2, 2, '-']]
(4, 2, 1, 3): [[0, '-', '-